In [1]:
# conding: utf-8

from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *

import json

In [2]:
from pprint import pprint
from sympy.interactive import printing
printing.init_printing()

In [3]:
"""
    Este trecho de código é necessário apenas quando o notebook é executado em ambiente local
"""

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
try:
    sc = SparkContext('local')
    spark = SparkSession(sc)
except ValueError:
    pass  # a spark context is already created

### Recuperando schema interface

In [4]:
# structured_jsons_path = '../../../../aws_s3/teste/structured_json/checkout/00_CheckoutOrder/001E600A63944702BF4EA7A92FCD3833/id/*'
structured_jsons_path = '../../../../data/s3/checkout/structured_json/{*}CheckoutOrder/*/id/*'
structured_df = spark.read.json(structured_jsons_path)

schema = structured_df.schema

### Lendo dados

In [5]:
# history_datapath = '../../../../aws_s3/teste/history/checkout/001E600A63944702BF4EA7A92FCD3833/id/*'
history_datapath = '../../../../data/s3/checkout/history/YY_CheckoutOrder/*/id/*'
df = spark.read.json(history_datapath)

### Código para converter os dados com spark usando cast

#### Casting de Items

In [ ]:
# Get Item Type
types = filter(lambda f: f.name == "Items", structured_df.schema.fields)
itemType = types[0].dataType

def convert_item(data):
    def _parse_product_categories(raw_product_categories):
        product_categories = []
        for k,v in raw_product_categories.items():
            product_categories.append({
                'id': k,
                'name': v
            })
        return product_categories
    
    items = json.loads(data)
    for item in items:
        product_categories = _parse_product_categories(item.get('productCategories', {}))
        item['productCategories'] = product_categories

    return items

# Register functions as Spark UDFs 
udf_getData = UserDefinedFunction(convert_item, itemType)

df = df.withColumn('Items', udf_getData("Items"))

In [ ]:
df.select("Items").show(5)

#### Casting de ItemMetadata

In [ ]:
types = filter(lambda f: f.name == "ItemMetadata", structured_df.schema.fields)
item_metadata_type = types[0].dataType

def convert_item_metadata(data):
    if data:
        itemmetadata = json.loads(data)
        len_items_itemmetadata = len(itemmetadata["items"])
        for i in range(len_items_itemmetadata):
            if ("assemblyOptions" in itemmetadata["items"][i]):
                del itemmetadata["items"][i]["assemblyOptions"]
        return itemmetadata
        
# Register functions as Spark UDFs 
udf_convert_item_metadata = UserDefinedFunction(convert_item_metadata, item_metadata_type)

df = df.withColumn("ItemMetadata", udf_convert_item_metadata("ItemMetadata"))

In [ ]:
df.select("ItemMetadata").show(5)

#### Casting RateAndBenefits

In [ ]:
types = filter(lambda f: f.name == "RatesAndBenefitsData", structured_df.schema.fields)
rateandbenefits_type = types[0].dataType

def convert_ratesandbenefits(data):
    KEY_IDENTIFIERS = "rateAndBenefitsIdentifiers"
    KEY_MATCH_PARAMS = "matchedParameters"
    KEY_ADDINFO = "additionalInfo"
    data = json.loads(data)
    if data and KEY_IDENTIFIERS in data:
        for i in range(len(data[KEY_IDENTIFIERS])):
            if KEY_MATCH_PARAMS in data[KEY_IDENTIFIERS][i]:
                del data[KEY_IDENTIFIERS][i][KEY_MATCH_PARAMS]
            if KEY_ADDINFO in data[KEY_IDENTIFIERS][i]:
                del data[KEY_IDENTIFIERS][i][KEY_ADDINFO]
    return data
        
# Register functions as Spark UDFs 
udf_convert_ratesandbenefits = UserDefinedFunction(convert_ratesandbenefits, rateandbenefits_type)

df = df.withColumn("RatesAndBenefitsData", udf_convert_ratesandbenefits("RatesAndBenefitsData"))

In [ ]:
df.select("RatesAndBenefitsData").show(1, False)

In [ ]:
df.select("RatesAndBenefitsData").collect()

#### Casting CustomData

In [ ]:
from pyspark.sql.utils import AnalysisException
try:
    df.where("CustomData is not null").select("CustomData").count()
except AnalysisException:
    # there is no col CustomData in data
    pass

In [ ]:
types = filter(lambda f: f.name == "CustomData", structured_df.schema.fields)
customdata_type = types[0].dataType

def convert_customdata(data):
    KEY_CUSTOMAPP = "customApps"
    KEY_FIELDS = "fields"
    KEY_EXTRA_CONTENT = "cart-extra-context"
    
    customdata = data and json.loads(data)
    if customdata and KEY_CUSTOMAPP in customdata:
        for i in range(len(customdata[KEY_CUSTOMAPP])):
            if KEY_FIELDS in customdata[KEY_CUSTOMAPP][i] and\
                KEY_EXTRA_CONTENT in customdata[KEY_CUSTOMAPP][i][KEY_FIELDS]:
                    del customdata[KEY_CUSTOMAPP][i][KEY_FIELDS][KEY_EXTRA_CONTENT]
    return customdata
        
# Register functions as Spark UDFs 
udf_convert_customdata = UserDefinedFunction(convert_customdata, customdata_type)

df = df.withColumn("CustomData", udf_convert_customdata("CustomData"))

In [ ]:
df.select("CustomData").show(5)

#### Casting Attachment

In [6]:
ATTACHMENT = "attachment"

def remove_attachments(dic):    
    dic_copy = dic.copy()
    for key in dic_copy:
        if(ATTACHMENT in key.lower()):
            del dic[key]
        elif(type(dic_copy[key]) == dict):
            remove_attachments(dic[key])
        elif(type(dic_copy[key]) == list):
            for item in dic_copy[key]:
                if(type(item) == dict):
                    remove_attachments(item)
    return dic
    
def get_transform_data(field):
    if field is not str: return
    structuted_field = field and json.loads(field)
    if type(structuted_field) is dict:
        structuted_field = remove_attachments(structuted_field)
    return structuted_field
    
def transform_struct_df(df_no_structured, fields):
    for field in fields:
        field_type = field.dataType
        field_name = field.name

        if ATTACHMENT in field_name.lower():
            df_no_structured = df_no_structured.drop(field_name)
        elif field_type != StringType():
            udf_get_transform_data = UserDefinedFunction(lambda d: get_transform_data(d), field_type)
            df_no_structured = df_no_structured.withColumn(field_name, udf_get_transform_data(field_name))
    
    return df_no_structured

In [7]:
df = transform_struct_df(df, df.schema.fields)

In [8]:
df.show(1, False)

+-----------+------------------+----------+----------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------+-------------------+---------------+-----------------------------------------------+----------------+--------+---------------+-----------------+--------------------------------+-----------+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
df.collect()

[Row(AffiliateId=None, CallCenterOperator=None, ChangeData=None, ClientPreferencesData=u'{"locale":"pt-BR","optinNewsLetter":true,"id":"clientPreferencesData"}', ClientProfileData=u'{"id":"clientProfileData","email":"allan.lemos.sp@gmail.com","firstName":"Allan","lastName":"Lemos","document":"32647257892","phone":"+5511987542219","corporateName":null,"tradeName":null,"corporateDocument":null,"stateInscription":null,"corporatePhone":null,"isCorporate":false,"documentType":"cpf"}', CreationDate=u'2015-07-17T17:57:42.1911258Z', CreationEnvironment=u'STABLE', CreationVersion=u'1.101.78.1106', FollowUpEmail=u'4851ca43f44847b9a70f60f408c11cbf@ct.vtex.com.br', GiftRegistryData=None, HostName=u'lekssa', Id=u'547463458382-01', IgnoreProfileData=None, InstanceId=u'00F582D386DA4F35B7F83ED8DB744B9E', IsCompleted=None, IsToSavePersonData=None, Items=u'[{"uniqueId":"424DD0B27DEF46AA84AD8FC70D8EBEF8","id":"101","sellerSku":"101","productId":"101","lockId":null,"availability":"available","name":"JOGO 

#### Particionando dados e escrevendo

In [ ]:
def getYear(lastChange, creationDate):
    date = lastChange if lastChange is not None else creationDate
    return date.split('T')[0].split('-')[0]

def getMonth(lastChange, creationDate):
    date = lastChange if lastChange is not None else creationDate
    return date.split('T')[0].split('-')[1]

def getDay(lastChange, creationDate):
    date = lastChange if lastChange is not None else creationDate
    return date.split('T')[0].split('-')[2]

# Register functions as Spark UDFs 
udf_getYear = UserDefinedFunction(getYear, StringType())
udf_getMonth = UserDefinedFunction(getMonth, StringType())
udf_getDay = UserDefinedFunction(getDay, StringType())

In [ ]:
# Create the Columns for the Partitions
df = df.withColumn('YEAR', udf_getYear(df.LastChange, df.CreationDate))
df = df.withColumn('MONTH', udf_getMonth(df.LastChange, df.CreationDate))
df = df.withColumn('DAY', udf_getDay(df.LastChange, df.CreationDate))

In [ ]:
# Save table to S3 using Parquet format and partitioning by defined columns
df.write.partitionBy(['YEAR','MONTH','DAY','InstanceId'])\
    .mode('append')\
    .json('../../../../blablabla/')
#     .parquet('s3://vtex.datalake/consumable_tables/')